# 1. Introduction

- Chose to use more data (option 2: "Share and grab more data"). Note that the data in the provided OneDrive folder was available under a [Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/) license.
- 


# 2. Model training and Results
Any code to train a model and produce results or a reference to the notebook/code file.

In [ ]:
# ran from colab

!pip install -q fastbook

import fastbook
fastbook.setup_book()

from fastai.vision.all import *

path = Path('/content/gdrive/MyDrive/ColabNotebooks/digits')

seed_val = 25
set_seed(seed_val)

def get_parent_as_list(x): return [parent_label(x)]
fingers = DataBlock(
    blocks=(ImageBlock, CategoryBlock), #image->discrete category
    # blocks=(ImageBlock, MultiCategoryBlock), #image->discrete category
    get_items=get_image_files, # path->images
    splitter=GrandparentSplitter(train_name='train', valid_name='valid'), #40 training, 10 validation images per finger
    get_y=parent_label, # get the label from the directory name
    # get_y=get_parent_as_list, # get the label from the directory name
    item_tfms=Resize(640, ResizeMethod.Squish), # resize to 640x640 px; use Squish so fingers aren't cropped
    batch_tfms=aug_transforms(size=420, flip_vert=False, max_zoom=1.0, min_zoom=1.0, max_rotate=5)) #a set of transformations like  rotation, flipping, perspective warping, brightness changes and contrast changes

dls = fingers.dataloaders(path, bs=32)

dls.train.show_batch(unique=True)
dls.valid.show_batch(unique=True)

set_seed(seed_val)

learn = cnn_learner(dls, resnet18, metrics=accuracy)

lr_min, lr_steep, lr_valley, lr_slide = learn.lr_find(suggest_funcs=(minimum, steep, valley, slide))

# chose lr_min instead of lr_steep because steep was on the very far left side of the curve and produced poor results (<20% accuracy)
learn.fine_tune(6, base_lr=lr_min)

learn.export(path/'../lab4_finger_count_fine_tuned.pkl')

# 3. Summary and Conclusion
Summarize your new results and state your lab3 and lab4 results.


Conclusion: How large is the improvement in the performance of the classifier due to the approach chosen? Include your interpretation why it worked (or did not work).

# 4. Reflection
Include a sentence or two about
- what you liked or disliked,
- found interesting, confusing, challangeing, motivating

while working on this assignment.

*YOUR ANSWER HERE*

